In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sortedcontainers import SortedList
import math

Tách thành tập train và test

In [7]:
df = pd.read_csv('../dataset/rating_final.csv')
df = shuffle(df)

cut_off_length = int(0.8 * len(df))
df_train = df[:cut_off_length]
df_test = df[cut_off_length:]

In [9]:
user_to_movie = {}
movie_to_user = {}
movie_user_rating = {}

for row in df_train.values:
    movie_id, user_id, rate = row
    if movie_id not in user_to_movie:
        user_to_movie[user_id] = [movie_id]
    else:
        user_to_movie[user_id].append(movie_id)
    
    if movie_id not in movie_to_user:
        movie_to_user[movie_id] = [user_id]
    else:
        movie_to_user[movie_id].append(user_id)

    movie_user_rating[(movie_id, user_id)] = rate

movie_user_rating


{'ur15794099': ['tt7322224'],
 'ur18374320': ['tt0029583'],
 'ur48675674': ['tt1745960'],
 'ur4445210': ['tt8579674'],
 'ur21319675': ['tt3262342'],
 'ur3670492': ['tt0092067'],
 'ur44965663': ['tt8760708'],
 'ur23656818': ['tt7888964'],
 'ur1173088': ['tt0112130'],
 'ur13887584': ['tt0114369'],
 'ur1609501': ['tt0468569'],
 'ur19892172': ['tt0325980'],
 'ur0622275': ['tt0082971'],
 'ur41031967': ['tt11145118'],
 'ur23255110': ['tt0048473'],
 'ur107065132': ['tt2861424'],
 'ur44588015': ['tt0107048'],
 'ur25552762': ['tt7562112'],
 'ur1234929': ['tt0088247'],
 'ur0491610': ['tt0083658'],
 'ur3270789': ['tt1049413'],
 'ur82541984': ['tt1745960'],
 'ur59565842': ['tt1486217'],
 'ur2860723': ['tt0096697'],
 'ur19117722': ['tt1291584'],
 'ur2483625': ['tt0068646'],
 'ur55782634': ['tt2119532'],
 'ur13566917': ['tt0071315'],
 'ur0715971': ['tt0060196'],
 'ur33374263': ['tt3011894'],
 'ur66106173': ['tt5198068'],
 'ur107418055': ['tt0386676'],
 'ur48102834': ['tt11032374'],
 'ur77131575': ['

In [4]:
movie_user_rating_test = {}
for row in df_test.values:
    movie_id, user_id, rate = row

    movie_user_rating_test[(movie_id, user_id)] = rate

In [5]:
number_of_movie = int(len(movie_to_user.keys()))
number_of_user = int(len(user_to_movie.keys()))

print(f'Number of movie: {number_of_movie}')
print(f'Number of user: {number_of_user}')

Number of movie: 490
Number of user: 1000


In [6]:
k_neighbor = 20
limit_neighbor = 5
neighbors = {}
averages = {}

for movie_id_i in movie_to_user.keys():
    # get all user who rated for movie_id_i
    user_rated_movie_i = movie_to_user[movie_id_i]

    # get all rating of users gave for movie_id_i
    rating_user_for_movie_i = [movie_user_rating[(movie_id_i, user_id)] for user_id in user_rated_movie_i]

    # calculate avarage rating of all user gave for movie_id_i
    avg_rating_movie_i = np.mean(rating_user_for_movie_i)

    averages[movie_id] = avg_rating_movie_i

    sorted_list = SortedList()
    for movie_id_j in movie_to_user.keys():
        if movie_id_j == movie_id_i:
            continue
        
        # get all user who rated for movie_id_j
        user_rated_movie_j = movie_to_user[movie_id_j]

        common_users = list(set(user_rated_movie_i).intersection(set(user_rated_movie_j)))

        if len(common_users) < limit_neighbor:
            continue
        
        # get all rating of users gave for movie_id_j
        rating_user_for_movie_j = [movie_user_rating[(movie_id_j, user_id)] for user_id in user_rated_movie_j]

        # calculate avarage rating of all user gave for movie_id_j
        avg_rating_movie_j = np.mean(rating_user_for_movie_j)

        deviation = 0
        sigmoid_i = 0
        sigmoid_j = 0
        for user_id in common_users:
            dev_i = (movie_user_rating[(movie_id_i, user_id)] - avg_rating_movie_i)
            dev_j = (movie_user_rating[(movie_id_j, user_id)] - avg_rating_movie_j)
            deviation += dev_i * dev_j
            sigmoid_i += dev_i * dev_i
            sigmoid_j += dev_j * dev_j

        s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))
        sorted_list.add(s_ij)
        if len(sorted_list) > k_neighbor:
            del sorted_list[0]

    neighbors[movie_id_i] = sorted_list
print(neighbors)

{'tt2085059': SortedList([0.6589297479405742, 0.6617177922777975, 0.6750903416886344, 0.6780512420472894, 0.678426684902794, 0.6817475750926327, 0.6824238519221065, 0.6863430396887827, 0.6913378939141802, 0.6923265304330882, 0.6957712808890328, 0.6963627405197375, 0.703053158820145, 0.7256563488403529, 0.8227802228366947, 0.8252203955147452, 0.8354128752697707, 0.8461375553627591, 0.8774411189778818, 0.9099639210759316]), 'tt10640346': SortedList([0.65608163653165, 0.6571062324515008, 0.6586619318329, 0.6681503825554728, 0.6691467926208836, 0.6862344525804913, 0.689274925094169, 0.7026293706903616, 0.7253374889094436, 0.7658367434868844, 0.7845260123674698, 0.8146036437555119, 0.826557680307092, 0.8541903485316263, 0.8782734604207889, 0.8793148229485988, 0.9214864880391074, 0.9487108171283049, 0.9589010427886191, 0.9762551410155071]), 'tt0071315': SortedList([0.655285432096707, 0.6557240341076518, 0.657836490631519, 0.6589297479405742, 0.6653982326029904, 0.6697773263164829, 0.67393295